# Beispiel-Implementierung: Lokale Suchmaschine

## Ziel der Beispiel-Implementierung
Im Folgenden wird eine Anwendung der zuvor theoretisch diskutierten Inhalte vorgestellt. Dabei soll eine lokale Suchmaschine entwickelt werden, welche in der Lage ist, pdf-Dateien auf einem lokalen Computer-System zu parsen, in einen invertierten Index aufzunehmen sowie Suchanfragen eines Benutzers sinnvoll zu beantworten. <br>
Zur Relevanz-Bestimmung der Dokumente wird das TF-IDF-Maß, welches bereits vorgestellt wurde, genutzt. Um den Index zu speichern, wird die von Python mitgelieferte Datenstruktur "dictionary", welche im Grunde eine Hashmap ist, genutzt.
Weiter werden einige Bibliotheken eingesetzt, welche einige Vorarbeit leisten und damit den Code der Beispiel-Implementierung auf das Wesentliche beschränken. So soll die grundlegende Arbeitsweise eines Information Retrieval-Systems dargelegt werden.

## Genutzte Bibliotheken
Bevor mit der eigentlichen Implementierung der lokalen Suchmaschine begonnen werden kann, müssen einige Bibliotheken eingebunden werden. Darunter fallen Apache Tika, das Math-Modul von Python, os (um auf die Directories zugreifen zu können), python-magic, regular expressions (re) (und noch weitere, bei Bedarf einfügen!). <br>

### Tika
Tika liefert eine Parser, mit dessen Hilfe der Text aus - unter anderem - pdf-Dateien extrahiert werden kann.
Mit dem Aufruf _parser.from_\__file(file)_ kann eine pdf-Datei in reinen Text umgewandelt werden. Die Funktion liefert ein Dictionary zurück, welches einen Key _content_ besitzt, über den auf den Inhalt der pdf-Datei zugegriffen werden kann.

### python-magic
Mittels python-magic ist es möglich, unabhängig von der Dateiendung, den Typ einer Datei zu ermitteln. Dies hat den Vorteil, dass die Suchmaschine sowohl unter Windows, als auch unter Unix-Systemen, alle pdf-Dateien finden kann, da unter Unix die Dateiendung keine garantierten Rückschlüsse auf den Typ der Datei zulässt.


In [143]:
from tika import parser
import magic
import math
import os
import string
import re
from nltk.tokenize import RegexpTokenizer

## Die Document-Klasse
Das Speichern er für das Retrieval wichtigen Informationen, geschieht mittels einer Document-Klasse. Diese Klasse hält alle Attribute, die wichtig sind, um das TF-IDF-Maß berechnen zu können. Diese Attribute sind:
- url
- length
- id

Die Variable _url_ ist ein String und enthält den Pfad zum Dokument, welches durch das entsprechende Document-Objekt repräsentiert wird. _length_ ist ein Integer und beinhaltet die Anzahl der Wörter, die in dem Dokument vorkommen und _id_ ist die eindeutige Dokumenten-ID, zu der weiter unten noch genaueres gesagt wird.

In [186]:
class Document:
    def __init__(self, url, length, id):
        self.url = url
        self.length = length
        self.id = id

## Der Index
Nachdem die benötigten Bibliotheken bekannt sind, kann der Index implementiert werden. Bevor dieser jedoch aufgebaut werden kann, sind einige Vorarbeiten nötig, die durch die vorgestellten Bibliotheken gestützt werden.
Der Index wird im Folgenden als Klasse implementiert. Diese beinhaltet die folgenden Methoden, die in den folgenden Abschnitten genauer diskutiert werden:
- buildIndex()
- retrieve()
- calcTFIDF()

Weiter werden die folgenden Member-Variablen benötigt:
- hashmap
- fileCount
- docHashmap

Die Member-Variable _hashmap_ ordnet allen Termen eine Liste von eindeutigen Dokumenten-IDs zu, in denen sie vorkommen. Im Dictionary _docHashmap_ werden die Dokuemnten-IDs als Key genutzt, um eine Zurodnung von Dokumenten-IDs auf Document-Objekte zu ermöglichen. Die Variable _fileCount_ ist ein Integer und wird für jedes gefundene Dokument um _1_ hochgezählt. Damit ist diese Variable qualifiziert als eindeutige Dokumenten-ID zu fungieren, wofür sie genutzt wird.


In [187]:
class Index:
    hashmap = {} #dictionary
    fileCount = 0 #integer, Gesamtzahl aller gefunden Dateien
    docHashmap = {}

## buildIndex
Die Methode _buildIndex_ baut - wie der Name bereits vermuten lässt - den Index auf. Dabei dient ein Dictionary als Basis-Datenstruktur. Zudem führt diese Methode die Verarbeitung der pdf-Dateien mittels Apache tika und python-magic durch.

Der erste Schritt stellt das Iterieren über alle Directories dar. Gestartet wird bei Linux-Systemen im Root-Directory, unter Windows-Systemen muss über jede Partition iteriert werden.

In [188]:
def buildIndex(self):
    # alle Start-Verzeichnisse holen
    #start = self.__getStartDirectories()
    start = ["F:/Jonas/Uni/Arbeiten/Information Retrieval/Information-Retrieval/Latex"]
    # Magic-Instanz erstellen, um Datei-Typ bestimmen zu können
    mime = magic.Magic(mime=True)
    
    for s in start:
        for subdir, dir, files in os.walk(s):
            for f in files:
                path = s+"/"+f
                if mime.from_file(path) == "application/pdf":
                    # in Text umwawndeln und tokenization durchführen
                    fileData = parser.from_file(s+"/"+f)
                    rawText = fileData['content']
                    self.fileCount += 1
                    
                    processedText = self.__preprocessText(rawText)
                    document = Document(path, len(processedText), self.fileCount)
                    self.docHashmap.update({self.fileCount : document})
                    self.__addToIndex(self.fileCount, processedText)
                    
    return

Index.buildIndex = buildIndex

### Hilfsmethoden
In diesem Abschnitt werden die genutzten Hilfsmethoden kurz vorgestellt. Diese werden jedoch nicht in der Tiefe behandelt, wie die drei Haupt-Methoden behandelt werden. 

#### __getStartDirectories
Die Methode _\_\_getSartDirectories_ liefert eine Liste zurück, welche abhängig vom Betriebssystem, auf dem die Suchmaschine läuft, die Start-Verzeichnisse zurückgibt, in denen nach pdf-Dateien gesucht werden soll. Falls das zugrunde liegende Betriebssystem ein Linux-basiertes System ist, wird die Liste __["/"]__ zurückgegeben, falls ein Windows-System zugrundeliegt, wird die Liste aller Partitionen zurückgegeben.

In [189]:
def __getStartDirectories(self):
    start = []
    
    if platform.system() == "Linux":
        start.append("/")
    elif platform.system() == "Windows":
        start = ['%s:' % d for d in string.ascii_uppercase if os.path.exists('%s:' % d)]
    else:
        raise EnvironmentError
        
    return start

Index.__getStartDirectories = __getStartDirectories

#### __addToIndex

Diese Methode bekommt als Argument einen String, welcher den von tika extrahierten Text enthält. Dieser String wird mithilfe der in Python enthaltenen Methode _split_ aufgeteilt und in einer Liste zusammengefasst. Weiter werden Satzzeichen wie Punkte, Kommata, etc. aus dem String bzw. der Liste herausgefiltert.

In [190]:
def __addToIndex(self, document, terms):
    for t in terms:
        try:
            docs = self.hashmap[t]
            if document not in docs:
                docs.append(document)
                self.hashmap.update({t : docs})
        except KeyError:
            docs = [document]
            self.hashmap.update({t : docs})
    
Index.__addToIndex = __addToIndex

#### __preprocessText
Diese Methode dient der Vorverarbeitung der Texte, die in den pdf-Dokumenten stehen. Als erster Schritt wird der gesamte Text in Lower-Case gesetzt, damit bei der Suche später die Groß- bzw. Kleinschreibung unwichtig ist.
Im nächsten Schritt werden alle Zahlen aus dem Text entfernt.
Innerhalb der for-Schleife werden Trennungen von Wörtern, die auf zwei Zeilen aufgeteilt wurden, wieder zusammengefügt. Als letzter Schritt werden mithilfe des von _nltk_ migelieferten _RegexpTokenizer_ Tokens gebildet. Diese Tokens entsprechen hier beriets den Termen, die dann im Dictionary des Index aufgenommen werden.

In [191]:
def __preprocessText(self, txt):
    # lower all:
    txt = txt.lower()
    
    # remove digits
    txt = re.sub(r'\d+', '', txt)
            
    # tokenize the text
    tokenizer = RegexpTokenizer(r'[a-zA-Z]+-$|\w+')
    result = tokenizer.tokenize(txt)
    
    # concatenate divided words
    for word in result:
        if word[-1] == '-':
            ind = result.index(word)
            corrected = word[:-1]+result[ind+1]
            result[ind] = corrected
            del result[ind+1]
    return result
    
Index.__preprocessText = __preprocessText

#### retrieve
Die retrieve-Methode dient der Suche. Die Idee dabei ist, dass der Nutzer ein oder mehrere Schlagworte eingeben kann, auf deren Basis die am besten passenden Dokumente zurückgeliefert werden. Auch die Schlagworte werden den gleichen Normalisierungs-Prozess durchlaufen wie die Texte der Dokumente.

Diese Methode ist - im Gegensatz zu den vorherigen Methoden - mengenbasiert. Dies hat folgenden Hintergrund: Wenn der Nutzer mehrere Schlagworte eingibt, ist es möglich, dass mehrere Schlagworte in den gleichen Dokumenten vorkommen. Damit die Dokumente nicht doppelt in der Ergebnis-Liste vorkommen, werden Mengen verwendet, da in Mengen per Definition jedes Element nur ein Mal enthalten sein darf.

Die Implementierung dieser Methode ist hier zu sehen:

In [192]:
def retrieve(self, searchString):
    # pre-processing
    processedStrings = self.__preprocessText(searchString)
    result = set()
    for word in processedStrings:
        documents = set(self.hashmap[word])
        result = result.union(documents)
        
    return result

Index.retrieve = retrieve

In [193]:
ind = Index()
ind.buildIndex()

In [194]:
print(ind.retrieve("Information"))
print(ind.docHashmap[1].url)

{1}
F:/Jonas/Uni/Arbeiten/Information Retrieval/Information-Retrieval/Latex/main.pdf
